In [1]:
import psycopg2
import pyodbc
import pandas as pd

In [11]:
def get_db_cursor():
    username = "doadmin"
    password = 'yzmodwh2oh16iks6'
    host = 'db-postgresql-cl1-do-user-2276924-0.db.ondigitalocean.com'
    port = 25060
    database = 'MandatoryMetadata'
    schema = 'historical'

    conn = psycopg2.connect(host=host,database=database, user=username, password=password, port=port)
    cur = conn.cursor()
    cur.execute("SET search_path TO " + schema)

    return conn, cur

In [12]:
conn, cur = get_db_cursor()

In [13]:
sample_codes = pd.read_csv("outputFiles/joined_spectra_with_lims.csv")['sample_code'].values

In [14]:
predictions = pd.read_sql(f"select * from prediction p where sample_code in {tuple(sample_codes)}", con=conn)

C:\Users\Tsuma Thomas\AppData\Roaming\Python\Python310\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [16]:
chemicals = pd.read_sql("select * from chemical c", con=conn)

C:\Users\Tsuma Thomas\AppData\Roaming\Python\Python310\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [17]:
predictions = pd.merge(predictions, chemicals, on='chemical_id', how='left')[['sample_code','chemical_name','prediction']]

In [18]:
predictions

,sample_code,chemical_name,prediction
0,FA419-43SA0001,organic_carbon,2.438334
1,FA419-46SA0003,organic_carbon,2.393521
2,FA419-45SA0002,organic_carbon,1.989184
3,FA611-430SA0001,organic_carbon,2.310375
4,FA419-47SA0005,organic_carbon,2.620643
...,...,...,...
2053,FA326-64SA0004,silt,12.482641
2054,FA326-64SA0003,silt,11.769399
2055,FA326-64SA0005,clay,20.376280
2056,FA326-64SA0004,clay,37.785984


In [19]:
predictions = pd.pivot_table(predictions, values='prediction', index=['sample_code'], columns=['chemical_name'], aggfunc='max')

In [20]:
predictions

chemical_name,aluminium,boron,calcium,cec,clay,copper,exchangeable_acidity,iron,magnesium,manganese,organic_carbon,ph,phosphorus,potassium,sand,silt,sodium,sulphur,total_nitrogen,zinc
sample_code,,,,,,,,,,,,,,,,,,,,
CB278SA0006,1060.749268,2.446498,3460.958740,32.242165,37.126476,3.798010,0.146646,185.037827,534.957703,250.842377,2.478024,6.758463,89.531334,1423.111572,32.402817,24.352398,80.510551,62.841133,0.202751,10.984707
CB278SA0007,1137.127197,0.769150,2213.581299,22.815117,29.589132,3.292969,0.251813,169.457962,404.537109,230.326813,1.510033,5.631011,38.429367,824.537781,31.291742,23.391790,80.183617,28.878576,0.131238,7.858414
CB278SA0008,1109.127441,1.408169,2482.500732,24.380295,39.040627,2.910913,0.206819,147.663040,419.980499,287.625580,2.068551,6.094843,41.586327,1007.039673,33.124012,18.250593,72.856422,33.375275,0.154644,8.171686
CC004SA0906,966.623901,1.193817,2994.062744,24.197252,48.920860,3.023098,0.200433,194.792801,411.047821,259.955658,2.334973,7.100417,82.768265,1208.451904,27.508087,24.735535,101.826225,39.911568,0.150163,10.952916
CC004SA0907,1030.695312,1.079055,2779.318604,24.146410,45.863167,2.944192,0.186376,203.519028,449.854797,203.538712,2.281078,6.318345,93.983032,1037.109741,29.279419,24.994850,109.089943,47.339977,0.150772,10.220988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FA807-765SA0006,705.676025,1.508906,4960.096191,34.209869,46.964447,3.589411,0.100604,168.129562,555.801086,287.880646,2.563360,7.181338,34.501621,1012.579285,31.482872,18.143147,157.048172,35.153671,0.179362,5.265543
FA807-765SA0007,602.855469,0.994529,2403.493652,18.995831,29.648222,2.701197,0.108921,205.493164,265.027802,173.123627,2.392315,6.908105,24.774557,302.122833,38.906414,25.516171,62.294586,24.229143,0.196919,4.306680
FA807-765SA0008,719.302185,0.776436,3180.481934,26.470648,38.136227,2.756223,0.233218,223.362152,339.771423,150.870743,2.782423,6.299331,39.795166,513.392700,36.561577,23.503082,89.142220,26.546789,0.198295,7.773492


In [21]:
predictions.to_csv("outputFiles/predictions.csv")